In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)  



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\Hannah\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [3]:
url = 'https://www.brewersassociation.org/directories/breweries/?location=GA'
browser.visit(url)

In [9]:
for i in range(4):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [12]:
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

# Convert the browser html to a soup object
html = browser.html
news_soup = soup(html, 'html.parser')

In [ ]:
slide_elem = news_soup.select_one('div.company-content')
slide_elem

In [ ]:
results = slide_elem.find('h3', class_='with-mini-hr')
results

In [ ]:
results.text

In [ ]:
#find all of the company content 
div_data = news_soup.find_all('div',class_='company-content')
div_data

In [13]:
company_names = news_soup.find_all('h3', class_='with-mini-hr')
company_names
len(company_names)

174

In [15]:
company_list = []
for i in company_names: 
    t = i.text
    company_list.append(t)
    
company_list

['52 Brewing',
 '6S Brewing Company',
 'Abide Brewing Company',
 'Akademia Brewing Company',
 'Amici At Trillith',
 'Amici Madison',
 'Anderby Brewing',
 'Angry Hops Brewing',
 'Anheuser-Busch Inc – Cartersville',
 'Arches Brewing',
 'Athentic Brewing Company',
 'Atlanta Brewing',
 'Awkward Brewing',
 'Back Paddle Brewing',
 'Barrier Island Brewing',
 'Beacon Brewing Co.',
 'Best End Brewing Company',
 'Big O Brewery',
 'Biggerstaff Brewing Company',
 'Black Bee Brewery',
 'Blackbird Farms Brewery',
 'BlueTarp Brewing Co',
 'Brackish Beer Company',
 'Braselton Brewing Company',
 'Brewery in Planning - Cumming, GA',
 'Bucking Goat Brewing Company',
 'Burnt Hickory Brewery',
 'Camp Brewing Company',
 'Candler Rail Brewery',
 'Cannon Brewpub',
 'Cartecay River Brewing Co',
 'Chattabrewchee Southern Brewhouse',
 'Cherokee Brewing + Pizza Company',
 'Cherry Street Brewing',
 'Cherry Street Brewpub At Halcyon',
 'Coastal Empire Beer Co',
 'Cochran Mill Brewing Company, LLC',
 'Contrast Artis

In [17]:
membership =news_soup.find_all('div', class_='logos')
membership

[<div class="logos"><img alt="" src="https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/ba_associate.png"/></div>,
 <div class="logos"><img alt="" src="https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/BA_member-1.png"/><img alt="" class="craft-logo" src="https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2018/12/BA_Independent_Seal-2.svg"/><div class="seal-popover"><div class="arrow-up"></div>There are many considerations when choosing a beer to enjoy. The independent craft brewer seal provides transparency for those who want to support small and independent U.S. craft brewers.</div></div>,
 <div class="logos"><img alt="" src="https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/BA_member-1.png"/><img alt="" class="craft-logo" src="https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2018/12/BA_Independent_Seal-2.svg"/><div class="seal-popover"><div class="arrow-up"></div>There are man

In [18]:
len(membership)

174

In [63]:
a = membership[1]
b= a.find_all('img')
b
for i in b: 
    print(i['src'])
    print(i['src']==member_str)

https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/BA_member-1.png
True
https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2018/12/BA_Independent_Seal-2.svg
False


In [57]:
member_type = []
craft =[] 

member_str = 'https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2019/05/BA_member-1.png'
craft_seal = 'https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2018/12/BA_Independent_Seal-2.svg'


for i in membership:
    mark= a.find_all('img')
    if len(mark)==2:
        temp = []
        for j in mark: 
            temp.append(j)
        if temp[0] == member_str:
            member_type.append('Beer Association Member')
            craft.append('Independent Craft Brewer')
        else: 
            member_type.append('Beer Association Associate Member')
            craft.append('Independent Craft Brewer')
    elif len(mark)==1:
        temp = []
        for j in mark: 
            temp.append(j)
        if temp == member_str:
            member_type.append('Beer Association Member')
        else: 
            member_type.append('Beer Association Associate Member')
    else: 
        member_type.append('NA')
        craft.append('NA')
    

In [65]:
craft

[]